# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

In [100]:
EPHYS_SAMPLE_RATE = 20000

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [16]:
# TRIAL_LABELS_DF = pd.read_excel("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")
TRIAL_LABELS_DF = pd.read_excel("../../data/rce_pilot_3_alone_comp_per_video_trial_labels.xlsx")

In [17]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX))

In [18]:
FULL_LFP_TRACES_PKL = "{}_10_per_trial_spectral_bans_sleap.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [31]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [18]:
LFP_SPECTRAL_DF.columns

Index(['video_name', 'tone_frame', 'tone_start_frame', 'reward_start',
       'reward_dispensed', 'tone_stop_frame', 'condition ',
       'competition_closeness', 'notes', '10s_before_tone_frame',
       ...
       'vHPC_lfp_RMS', 'vHPC_lfp_RMS_filtered', 'vHPC_lfp_mask',
       'vHPC_lfp_modified_zscore', 'vHPC_lfp_trace', 'vHPC_lfp_trace_filtered',
       'vHPC_mPFC_granger_all_frequencies_all_windows', 'vHPC_theta_band',
       'vHPC_theta_phase', 'video_timestamps'],
      dtype='object', length=220)

In [19]:
TRIAL_LABELS_DF.columns

Index(['Unnamed: 0', 'session_dir', 'tone_start_frame', 'reward_start',
       'reward_dispensed', 'tone_stop_frame', 'condition ',
       'competition_closeness', 'get_reward_frame', 'out_reward_frame',
       'notes', 'experiment', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'tone_start_timestamp',
       'tone_stop_timestamp', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps'],
      dtype='object')

## Merging the labels and LFP spectral

In [19]:
TRIAL_LABELS_DF.head()

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,0.0,80,100.0,74.0,NaN,NaN,NaN,NaN,...,[ 0 74],[ 0 74],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,0.0,75474.0,[ 0 75474],[ 0 75474]
1,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,1272.0,1352,1372.0,1471.0,NaN,NaN,NaN,NaN,...,[1282 1283] [1285 1297] [1326 1344] [1362 1509],[1326 1328] [1331 1347] [1380 1592],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,1275889.0,1475891.0,[1285888 1286688] [1287488 1300689] [1330286 1...,[1329489 1331487] [1334489 1350089] [1383490 1...
2,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,3268.0,3348,3368.0,3469.0,NaN,NaN,NaN,NaN,...,[3281 3324] [3324 3327] [3344 3347] [3347 3523],[3327 3704],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,3275911.0,3475914.0,[3287914 3330314] [3331114 3333712] [3351715 3...,[3333515 3712519]
3,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,4466.0,4546,4566.0,4667.0,NaN,NaN,NaN,NaN,...,[4441 4476] [4476 4489] [4508 4509] [4511 4534...,[4445 4473] [4494 4551] [4566 4830],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,4475927.0,4675931.0,[4450528 4485129] [4485929 4497529] [4516929 4...,[4454926 4483129] [4503531 4560730] [4575330 4...
4,0.0,20240320_142408_alone_comp_subj_3-1_and_3-3,5465.0,5545,5565.0,5664.0,NaN,NaN,NaN,NaN,...,[5483 5512] [5552 5698],[5455 5486] [5581 5794],20240320_142408_alone_comp_subj_3-1_and_3-3.1....,"['3.1', '3.3']",1830734.0,65425515.0,5475942.0,5675942.0,[5493742 5523542] [5563742 5709147],[5466941 5496741] [5592343 5806546]


In [20]:
LFP_SPECTRAL_DF.head()

,video_name,tone_frame,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,...,mPFC_lfp_trace,recording,session_path,subject,tone_frames,tone_timestamps,vHPC_lfp_trace,video_timestamps,current_timestamp,cluster_timestamp
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34322.0, 34322.0, 34322.0, 34322.0, 34322.0, ...","[34342.0, 34342.0, 34342.0, 34342.0, 34342.0, ...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...",...,"[4, -240, -358, -373, -650, -828, -870, -769, ...",20240320_171038_alone_comp_subj_4-2_t6b6_merged,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-43, -359, -511, -510, -808, -971, -1006, -86...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34322.0, 34322.0, 34322.0, 34322.0, 34322.0, ...","[34342.0, 34342.0, 34342.0, 34342.0, 34342.0, ...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...",...,"[72, 150, -75, -293, 95, 211, -296, -578, -414...",20240320_171038_alone_comp_subj_4-3_t5b5_merged,/scratch/back_up/reward_competition_extention/...,4.3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-4, -20, -265, -430, -102, 31, -350, -563, -4...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084..."
2,20240322_120625_alone_comp_subj_3-3_and_3-4.1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34349.0, 34349.0, 34349.0, 34349.0, 34349.0, ...","[34369.0, 34369.0, 34369.0, 34369.0, 34369.0, ...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...",...,"[-268, -429, 9, 379, -265, -470, 227, 795, 360...",20240322_120625_alone_comp_subj_3-4_t5b5_merged,/scratch/back_up/reward_competition_extention/...,3.4,"[[0, 101], [1299, 1500], [3297, 3496], [4494, ...","[[0, 102254], [1302671, 1502673], [3302693, 35...","[-222, -415, 8, 395, -252, -471, 220, 818, 430...","[1384, 1983, 2770, 4156, 5541, 5541, 6927, 831...","[34103476, 34104862, 34106248, 34106248, 34107...","[34103476, 34104862, 34106248, 34106248, 34107..."
3,20240322_120625_alone_comp_subj_3-3_and_3-4.1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34349.0, 34349.0, 34349.0, 34349.0, 34349.0, ...","[34369.0, 34369.0, 34369.0, 34369.0, 34369.0, ...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...",...,"[-195, 45, 75, -254, -341, -252, -154, -144, -...",20240322_120625_alone_comp_subj_3-3_t6b6_merged,/scratch/back_up/reward_competition_extention/...,3.3,"[[0, 101], [1299, 1500], [3297, 3496], [4494, ...","[[0, 102254], [1302671,

In [21]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))

In [23]:
LFP_SPECTRAL_DF["current_subject"] = LFP_SPECTRAL_DF["current_subject"].astype(str)


In [24]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1.videoTimeStamps.cameraHWSync',
       '20240320_142408_alone_comp_subj_3-1_and_3-3.2.videoTimeStamps.cameraHWSync',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1.videoTimeStamps.cameraHWSync',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.2.videoTimeStamps.cameraHWSync',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1.videoTimeStamps.cameraHWSync',
       nan,
       '20240322_120625_alone_comp_subj_3-3_and_3-4.2.videoTimeStamps.cameraHWSync',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1.videoTimeStamps.cameraHWSync',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.2.videoTimeStamps.cameraHWSync',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1.videoTimeStamps.cameraHWSync',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.2.videoTimeStamps.cameraHWSync',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1.videoTimeStamps.cameraHWSync',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.2

In [36]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["video_name"])

In [43]:
TRIAL_LABELS_DF = TRIAL_LABELS_DF.dropna(subset=["condition "])

In [44]:
TRIAL_LABELS_DF["video_name"] = TRIAL_LABELS_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [45]:
TRIAL_LABELS_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [46]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [47]:
TRIALS_AND_SPECTRAL_DF = pd.merge(TRIAL_LABELS_DF, LFP_SPECTRAL_DF, on=["video_name"], how="inner", suffixes=('', '_y'))

In [51]:
TRIALS_AND_SPECTRAL_DF.head()

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,mPFC_lfp_trace,recording,session_path,subject,tone_frames,tone_timestamps,vHPC_lfp_trace,video_timestamps,current_timestamp,cluster_timestamp
0,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,...,"[4, -240, -358, -373, -650, -828, -870, -769, ...",20240320_171038_alone_comp_subj_4-2_t6b6_merged,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-43, -359, -511, -510, -808, -971, -1006, -86...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084..."
1,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,...,"[72, 150, -75, -293, 95, 211, -296, -578, -414...",20240320_171038_alone_comp_subj_4-3_t5b5_merged,/scratch/back_up/reward_competition_extention/...,4.3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-4, -20, -265, -430, -102, 31, -350, -563, -4...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084..."
2,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,...,"[4, -240, -358, -373, -650, -828, -870, -769, ...",20240320_171038_alone_comp_subj_4-2_t6b6_merged,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-43, -359, -511, -510, -808, -971, -1006, -86...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084..."
3,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,...,"[72, 150, -75, -293, 95, 211, -296, -578, -414...",20240320_171038_alone_comp_subj_4-3_t5b5_merged,/scratch/back_up/reward_competition_extention/...,4.3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-4, -20, -265, -430, -102, 31, -350, -563, -4...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084..."
4,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,...,"[4, -240, -358, -373, -650, -828, -870, -769, ...",20240320_171038_alone_comp_subj_4-2_t6b6_merged,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-43, -359, -511, -510, -808, -971, -1006, -86...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084..."


In [52]:
TRIALS_AND_SPECTRAL_DF.tail()

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,mPFC_lfp_trace,recording,session_path,subject,tone_frames,tone_timestamps,vHPC_lfp_trace,video_timestamps,current_timestamp,cluster_timestamp
243,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,57880.0,57960,57980.0,58079.0,4.4,Subj 2 then Subj 1,NaN,NaN,...,"[7, 2, -97, -216, -198, -129, -103, -136, -196...",20240323_165815_alone_comp_subj_4-4_t6b6_merged,/scratch/back_up/reward_competition_extention/...,4.4,"[[0, 64], [1262, 1461], [3258, 3457], [4456, 4...","[[0, 63786], [1264201, 1464201], [3264226, 346...","[30, 17, -40, -97, -92, -77, -92, -122, -123, ...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186...","[36182281, 36183667, 36183667, 36185052, 36186..."
244,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,...,"[-78, -43, 18, 117, 150, -30, -244, -254, -169...",20240323_165815_alone_comp_subj_4-2_t5b5_merged,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 64], [1262, 1461], [3258, 3457], [4456, 4...","[[0, 63786], [1264201, 1464201], [3264226, 346...","[42, 159, 296, 359, 333, 124, -67, -62, 13, -3...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186...","[36182281, 36183667, 36183667, 36185052, 36186..."
245,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,...,"[7, 2, -97, -216, -198, -129, -103, -136, -196...",20240323_165815_alone_comp_subj_4-4_t6b6_merged,/scratch/back_up/reward_competition_extention/...,4.4,"[[0, 64], [1262, 1461], [3258, 3457], [4456, 4...","[[0, 63786], [1264201, 1464201], [3264226, 346...","[30, 17, -40, -97, -92, -77, -92, -122, -123, ...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186...","[36182281, 36183667, 36183667, 36185052, 36186..."
246,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,...,"[-78, -43, 18, 117, 150, -30, -244, -254, -169...",20240323_165815_alone_comp_subj_4-2_t5b5_merged,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 64], [1262, 1461], [3258, 3457], [4456, 4...","[[0, 63786], [1264201, 1464201], [3264226, 346...","[42, 159, 296, 359, 333, 124, -67, -62, 13, -3...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186...","[36182281, 36183667, 36183667, 36185052, 36186..."
247,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,61174.0,61254,61274.0,61375.0,tie,Close Call,NaN,NaN,...,"[7, 2, -97, -216, -198, -129, -103, -136, -196...",20240323_165815_alone_comp_subj_4-4_t6b6_merged,/scratch/back_up/reward_competition_extention/...,4.4,"[[0, 64], [1262, 1461], [3258, 3457], [4456, 4...","[[0, 63786], [1264201, 1464201], [3264226, 346...","[30, 17, -40, -97, -92, -77, -92, -122, -123, ...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186...","[36182281, 36183667, 36183667, 36185052, 36186..."


In [49]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=[col for col in TRIALS_AND_SPECTRAL_DF.columns if "_y" in col], errors="ignore")

In [72]:
timestamps_col = ["first_timestamp", "last_timestamp", "tone_start_timestamp", "tone_stop_timestamp"]

In [73]:
for col in timestamps_col:
    TRIALS_AND_SPECTRAL_DF[col] = TRIALS_AND_SPECTRAL_DF[col].astype(int)

# Calculating the relevant timestamps

In [101]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"] - 10 * EPHYS_SAMPLE_RATE

In [103]:
TRIALS_AND_SPECTRAL_DF["post_trial_end_timestamp"] = TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"] + 10 * EPHYS_SAMPLE_RATE

In [76]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

0    33680576
1    33680576
2    34680586
3    34680586
4    35780600
Name: baseline_start_timestamp, dtype: int64

In [77]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

0    33880576
1    33880576
2    34880586
3    34880586
4    35980600
Name: baseline_stop_timestamp, dtype: int64

In [78]:
TRIALS_AND_SPECTRAL_DF["tone_start_timestamp"].head()

0    34280576
1    34280576
2    35280586
3    35280586
4    36380600
Name: tone_start_timestamp, dtype: int64

In [79]:
TRIALS_AND_SPECTRAL_DF["tone_stop_timestamp"].head()

0    34480576
1    34480576
2    35480585
3    35480585
4    36580602
Name: tone_stop_timestamp, dtype: int64

In [80]:
TRIALS_AND_SPECTRAL_DF

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,session_path,subject,tone_frames,tone_timestamps,vHPC_lfp_trace,video_timestamps,current_timestamp,cluster_timestamp,baseline_start_timestamp,baseline_stop_timestamp
0,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,...,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-43, -359, -511, -510, -808, -971, -1006, -86...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084...",33680576,33880576
1,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,...,/scratch/back_up/reward_competition_extention/...,4.3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-4, -20, -265, -430, -102, 31, -350, -563, -4...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084...",33680576,33880576
2,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,...,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-43, -359, -511, -510, -808, -971, -1006, -86...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084...",34680586,34880586
3,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,...,/scratch/back_up/reward_competition_extention/...,4.3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-4, -20, -265, -430, -102, 31, -350, -563, -4...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084...",34680586,34880586
4,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,...,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79744], [1280158, 1480161], [3280184, 348...","[-43, -359, -511, -510, -808, -971, -1006, -86...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831...","[34079916, 34081302, 34082688, 34084074, 34084...","[34079916, 34081302, 34082688, 34084074, 34084...",35780600,35980600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,57880.0,57960,57980.0,58079.0,4.4,Subj 2 then Subj 1,NaN,NaN,...,/scratch/back_up/reward_competition_extention/...,4.4,"[[0, 64], [1262, 1461], [3258, 3457], [4456, 4...","[[0, 63786], [1264201, 1464201], [3264226, 346...","[30, 17, -40, -97, -92, -77, -92, -122, -123, ...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186...","[36182281, 36183667, 36183667, 36185052, 36186...",57364893,57564893
244,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,NaN,NaN,...,/scratch/back_up/reward_competition_extention/...,4.2,"[[0, 64], [1262, 1461], [3258, 3457], [4456, 4...","[[0, 63786], [1264201, 1464201], [3264226, 346...","[42, 159, 296, 359, 333, 124, -67, -62, 13, -3...","[1384, 2515, 2770, 4156, 5542, 5542, 6927, 831...","[36182281, 36183667, 36183667, 36185052, 36186...","[36182281, 36183667, 36183667, 36185052, 36186...",59564920,59764920
245,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,60077.0,60157,60177.0,60276.0,4.4,Subj 2 blocking Subj 1,

# Filtering out the SLEAP posed for during trials

In [81]:
TRIALS_AND_SPECTRAL_DF.columns

Index(['Unnamed: 0', 'session_dir', 'tone_start_frame', 'reward_start',
       'reward_dispensed', 'tone_stop_frame', 'condition ',
       'competition_closeness', 'get_reward_frame', 'out_reward_frame',
       'notes', 'experiment', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'all_subjects',
       'first_timestamp', 'last_timestamp', 'tone_start_timestamp',
       'tone_stop_timestamp', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'tone_frame', '10s_before_tone_frame',
       '10s_after_tone_frame', 'subject_thorax_to_agent_thorax',
       'nose_to_reward_port_sum', 'nose_to_reward_port_diff',
       'thorax_velocity_sum', 'thorax_velocity_diff',
       'to_reward_port_angle_sum', 'to_reward_port_angle_diff',
       'subject_nose_to_reward_port', 'subject_thorax_velocity',
       'subject_to_reward_port_angle', 'agent_nose_to_reward_port',
       'agent_thorax_velocity', 'agent_to_reward_port_angle', 'current_frame',
       'clos

In [52]:
TRIALS_AND_SPECTRAL_DF["video_frame"] = TRIALS_AND_SPECTRAL_DF["video_timestamps"].apply(lambda x: np.array(list(range(len(x)))) + 1)

In [53]:
TRIALS_AND_SPECTRAL_DF["video_frame"].head()

0    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
1    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
2    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
3    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
4    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
Name: video_frame, dtype: object

In [54]:
# sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "locations" in col or "velocity" in col or "to_reward_port" in col or "video_frame" in col]

In [91]:
sleap_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "thorax" in col or "nose" in col or "reward_port" in col or "standard_embedding_x" in col or "kmeans_cluster" in col or "current_frame" in col]

In [92]:
sleap_columns

['subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'current_frame',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'standard_embedding_x',
 'kmeans_cluster']

In [97]:
for col in sleap_columns:
    print(col)
    print(TRIALS_AND_SPECTRAL_DF[col].apply(lambda x: x.shape).iloc[0])

subject_thorax_to_agent_thorax
(12593,)
nose_to_reward_port_sum
(12593,)
nose_to_reward_port_diff
(12593,)
thorax_velocity_sum
(12593,)
thorax_velocity_diff
(12593,)
to_reward_port_angle_sum
(12593,)
to_reward_port_angle_diff
(12593,)
subject_nose_to_reward_port
(12593,)
subject_thorax_velocity
(12593,)
subject_to_reward_port_angle
(12593,)
agent_nose_to_reward_port
(12593,)
agent_thorax_velocity
(12593,)
agent_to_reward_port_angle
(12593,)
current_frame
(12593,)
closebool_subject_nose_to_reward_port
(12593,)
closebool_agent_nose_to_reward_port
(12593,)
movingbool_subject_thorax_velocity
(12593,)
movingbool_agent_thorax_velocity
(12593,)
standard_embedding_x
(12593,)
kmeans_cluster
(12593,)


In [ ]:
baseline_start_timestamp 
post_trial_end_timestamp

In [104]:
for col in sorted(sleap_columns):
    updated_item_col = "pre_and_post_{}".format(col)
    print(updated_item_col)
    updated_timestamp_col = "pre_and_post__video_timestamps"
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[1], axis=1)

TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["post_trial_end_timestamp"], timestamps=x["cluster_timestamp"], items=x[col])[0], axis=1)

pre_and_post_agent_nose_to_reward_port
pre_and_post_agent_thorax_velocity
pre_and_post_agent_to_reward_port_angle
pre_and_post_closebool_agent_nose_to_reward_port
pre_and_post_closebool_subject_nose_to_reward_port
pre_and_post_current_frame
pre_and_post_kmeans_cluster
pre_and_post_movingbool_agent_thorax_velocity
pre_and_post_movingbool_subject_thorax_velocity
pre_and_post_nose_to_reward_port_diff
pre_and_post_nose_to_reward_port_sum
pre_and_post_standard_embedding_x
pre_and_post_subject_nose_to_reward_port
pre_and_post_subject_thorax_to_agent_thorax
pre_and_post_subject_thorax_velocity
pre_and_post_subject_to_reward_port_angle
pre_and_post_thorax_velocity_diff
pre_and_post_thorax_velocity_sum
pre_and_post_to_reward_port_angle_diff
pre_and_post_to_reward_port_angle_sum


In [105]:
TRIALS_AND_SPECTRAL_DF

,Unnamed: 0,session_dir,tone_start_frame,reward_start,reward_dispensed,tone_stop_frame,condition,competition_closeness,get_reward_frame,out_reward_frame,...,pre_and_post_standard_embedding_x,pre_and_post_subject_nose_to_reward_port,pre_and_post_subject_thorax_to_agent_thorax,pre_and_post_subject_thorax_velocity,pre_and_post_subject_to_reward_port_angle,pre_and_post_thorax_velocity_diff,pre_and_post_thorax_velocity_sum,pre_and_post_to_reward_port_angle_diff,pre_and_post_to_reward_port_angle_sum,pre_and_post__video_timestamps
0,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,...,"[9.958147048950195, 9.980572700500488, 10.0073...","[2.5980964, 3.2073143, 3.9916637, 4.8566847, 5...","[4.9368286, 4.95431, 4.9935803, 5.1178784, 5.3...","[5.1050973, 5.082959, 5.072279, 5.091807, 5.14...","[2.4313493, 2.3651402, 2.2818353, 2.199549, 2....","[5.6331806, 5.7147374, 5.6083183, 5.3814564, 5...","[15.843375, 16.048643, 16.248707, 16.423662, 1...","[0.6321054, 0.6899271, 0.74678916, 0.802343, 0...","[4.8800473, 4.662217, 4.401958, 4.117365, 3.82...","[34081302, 34082688, 34084074, 34084074, 34085..."
1,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,34242.0,34322,34342.0,34442.0,4.3,After trial,NaN,NaN,...,"[9.958147048950195, 9.980572700500488, 10.0073...","[1.1949695, 0.93336076, 0.738537, 0.5684192, 0...","[4.9368286, 4.95431, 4.9935803, 5.1178784, 5.3...","[5.1050973, 5.082959, 5.072279, 5.091807, 5.14...","[2.4486978, 2.297077, 2.1201227, 1.9178162, 1....","[5.6331806, 5.7147374, 5.6083183, 5.3814564, 5...","[15.843375, 16.048643, 16.248707, 16.423662, 1...","[0.6321054, 0.6899271, 0.74678916, 0.802343, 0...","[4.8800473, 4.662217, 4.401958, 4.117365, 3.82...","[34081302, 34082688, 34084074, 34084074, 34085..."
2,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,...,"[5.59649133682251, 5.570141792297363, 5.584702...","[1.088969, 1.0757009, 1.0131229, 0.9762328, 0....","[24.583914, 23.842857, 23.053654, 22.21829, 21...","[0.84502804, 0.7296608, 0.62726414, 0.54466546...","[0.17591879, 0.17943281, 0.17603831, 0.2038250...","[15.349337, 18.314651, 21.41159, 24.472757, 27...","[17.039392, 19.773973, 22.66612, 25.562088, 28...","[1.9538506, 2.020497, 2.0900402, 2.0998404, 2....","[2.3056881, 2.3793628, 2.442117, 2.5074904, 2....","[35081900, 35081900, 35083286, 35084672, 35085..."
3,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,35241.0,35321,35341.0,35439.0,4.3,Subj 2 Only,NaN,NaN,...,"[5.59649133682251, 5.570141792297363, 5.584702...","[25.323456, 23.973078, 22.660395, 21.498892, 2...","[24.583914, 23.842857, 23.053654, 22.21829, 21...","[0.84502804, 0.7296608, 0.62726414, 0.54466546...","[2.1297693, 2.19993, 2.2660787, 2.3036654, 2.3...","[15.349337, 18.314651, 21.41159, 24.472757, 27...","[17.039392, 19.773973, 22.66612, 25.562088, 28...","[1.9538506, 2.020497, 2.0900402, 2.0998404, 2....","[2.3056881, 2.3793628, 2.442117, 2.5074904, 2....","[35081900, 35081900, 35083286, 35084672, 35085..."
4,4.0,20240320_171038_alone_comp_subj_4-2_and_4-3,36339.0,36419,36439.0,36538.0,4.3,Subj 2 blocking Subj 1,NaN,NaN,...,"[10.748048782348633, 10.608652114868164, 10.63...","[0.7897185, 0.7472973, 0.7605625, 0.8177425, 0...","[13.881315, 14.176304, 14.438505, 14.65004, 14...","[0.49317342, 0.4948187, 0.509421, 0.54049957, ...","[0.5789979, 0.5902529, 0.5984625, 0.60776705, ...","[5.934906, 6.0408955, 5.9854646, 5.8209095, 5....","[6.936893, 7.039064, 7.0063467, 6.898076, 6.69...","[0.3554064, 0.39349312, 0.42495528, 0.46008515...","[0.8359046, 0.79514295, 0.75987095, 0.7393834,...","[36180895, 36182281, 36183666, 36183666, 36186..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,24.0,20240323_165815_alone_comp_subj_4-2_and_4-4,57880.0,57960,57980.0,58079.0,4.4,Subj 2 then Subj 1,NaN,NaN,...,"[7.372586250305176, 7.445962905883789, 7.48498...","[28.46948, 28.584341, 28.669724, 28.858204, 29...","[24.5554

In [110]:
TRIALS_AND_SPECTRAL_DF["pre_and_post_current_frame"].iloc[0].shape

(599,)

In [116]:
TRIALS_AND_SPECTRAL_DF["pre_and_post_kmeans_cluster"].iloc[0][:20]

array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 6, 6, 5, 5, 2, 2, 2])

In [59]:
# for col in sorted(sleap_columns):
#     updated_item_col = "baseline_{}".format(col)
#     print(updated_item_col)
#     updated_timestamp_col = "baseline_video_timestamps".format(col)
#     if "agent" in col:
#         TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
#     else:
#         TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

# TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity
baseline_video_frame


In [111]:
# for col in sorted(sleap_columns):
#     updated_item_col = "trial_{}".format(col)
#     print(updated_item_col)
#     updated_timestamp_col = "trial_video_timestamps".format(col)
#     if "agent" in col:
#         TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1] if x["agent"] else np.nan, axis=1)
#     else:
#         TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[1], axis=1)

# TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["video_timestamps"], items=x[col])[0], axis=1)

In [61]:
TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["trial_subject_thorax_velocity"].apply(lambda x: np.isnan(x).any())]["baseline_subject_locations"]

Series([], Name: baseline_subject_locations, dtype: object)

In [62]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=sleap_columns + ["video_timestamps"], errors="ignore")

In [63]:
for col in TRIALS_AND_SPECTRAL_DF.columns:
    print(col)

tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band
mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_RMS_filtered_power_all_frequencies_all_windows
MD_RMS_filtered_power_all_frequencies_all_windows
LH_RMS_filtered_power_all_frequencies_all_windows
BLA_RMS_filtered_power_all_frequencies_all_windows
vHPC_RMS_filtered_power_all

In [ ]:
raise ValueError()

- Filtering coherence

In [64]:
coherence_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "coherence" in col and "timestamps" not in col and "calculation" not in col]

In [65]:
coherence_columns

['BLA_LH_RMS_filtered_coherence_all_frequencies_all_windows',
 'BLA_MD_RMS_filtered_coherence_all_frequencies_all_windows',
 'BLA_mPFC_RMS_filtered_coherence_all_frequencies_all_windows',
 'BLA_vHPC_RMS_filtered_coherence_all_frequencies_all_windows',
 'LH_MD_RMS_filtered_coherence_all_frequencies_all_windows',
 'LH_mPFC_RMS_filtered_coherence_all_frequencies_all_windows',
 'LH_vHPC_RMS_filtered_coherence_all_frequencies_all_windows',
 'MD_mPFC_RMS_filtered_coherence_all_frequencies_all_windows',
 'MD_vHPC_RMS_filtered_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_RMS_filtered_coherence_all_frequencies_all_windows',
 'BLA_LH_coherence_theta',
 'BLA_LH_coherence_gamma',
 'BLA_MD_coherence_theta',
 'BLA_MD_coherence_gamma',
 'BLA_mPFC_coherence_theta',
 'BLA_mPFC_coherence_gamma',
 'BLA_vHPC_coherence_theta',
 'BLA_vHPC_coherence_gamma',
 'LH_MD_coherence_theta',
 'LH_MD_coherence_gamma',
 'LH_mPFC_coherence_theta',
 'LH_mPFC_coherence_gamma',
 'LH_vHPC_coherence_theta',
 'LH_vHPC_

In [66]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

BLA_LH_RMS_filtered_coherence_all_frequencies
BLA_MD_RMS_filtered_coherence_all_frequencies
BLA_mPFC_RMS_filtered_coherence_all_frequencies
BLA_vHPC_RMS_filtered_coherence_all_frequencies
LH_MD_RMS_filtered_coherence_all_frequencies
LH_mPFC_RMS_filtered_coherence_all_frequencies
LH_vHPC_RMS_filtered_coherence_all_frequencies
MD_mPFC_RMS_filtered_coherence_all_frequencies
MD_vHPC_RMS_filtered_coherence_all_frequencies
mPFC_vHPC_RMS_filtered_coherence_all_frequencies
BLA_LH_coherence_theta
BLA_LH_coherence_gamma
BLA_MD_coherence_theta
BLA_MD_coherence_gamma
BLA_mPFC_coherence_theta
BLA_mPFC_coherence_gamma
BLA_vHPC_coherence_theta
BLA_vHPC_coherence_gamma
LH_MD_coherence_theta
LH_MD_coherence_gamma
LH_mPFC_coherence_theta
LH_mPFC_coherence_gamma
LH_vHPC_coherence_theta
LH_vHPC_coherence_gamma
MD_mPFC_coherence_theta
MD_mPFC_coherence_gamma
MD_vHPC_coherence_theta
MD_vHPC_coherence_gamma
mPFC_vHPC_coherence_theta
mPFC_vHPC_coherence_gamma


In [67]:
for col in coherence_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_coherence_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["coherence_timestamps"], items=x[col])[0], axis=1)

BLA_LH_RMS_filtered_coherence_all_frequencies
BLA_MD_RMS_filtered_coherence_all_frequencies
BLA_mPFC_RMS_filtered_coherence_all_frequencies
BLA_vHPC_RMS_filtered_coherence_all_frequencies
LH_MD_RMS_filtered_coherence_all_frequencies
LH_mPFC_RMS_filtered_coherence_all_frequencies
LH_vHPC_RMS_filtered_coherence_all_frequencies
MD_mPFC_RMS_filtered_coherence_all_frequencies
MD_vHPC_RMS_filtered_coherence_all_frequencies
mPFC_vHPC_RMS_filtered_coherence_all_frequencies
BLA_LH_coherence_theta
BLA_LH_coherence_gamma
BLA_MD_coherence_theta
BLA_MD_coherence_gamma
BLA_mPFC_coherence_theta
BLA_mPFC_coherence_gamma
BLA_vHPC_coherence_theta
BLA_vHPC_coherence_gamma
LH_MD_coherence_theta
LH_MD_coherence_gamma
LH_mPFC_coherence_theta
LH_mPFC_coherence_gamma
LH_vHPC_coherence_theta
LH_vHPC_coherence_gamma
MD_mPFC_coherence_theta
MD_mPFC_coherence_gamma
MD_vHPC_coherence_theta
MD_vHPC_coherence_gamma
mPFC_vHPC_coherence_theta
mPFC_vHPC_coherence_gamma


In [68]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=coherence_columns + ["coherence_timestamps"], errors="ignore")

- Filtering Grangers

In [69]:
granger_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "granger" in col and "timestamps" not in col and "calculation" not in col]

In [70]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows',
 'BLA-to-LH_granger_theta',
 'BLA-to-LH_

In [71]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

BLA_LH_granger_all_frequencies
LH_BLA_granger_all_frequencies
BLA_MD_granger_all_frequencies
MD_BLA_granger_all_frequencies
BLA_mPFC_granger_all_frequencies
mPFC_BLA_granger_all_frequencies
BLA_vHPC_granger_all_frequencies
vHPC_BLA_granger_all_frequencies
LH_MD_granger_all_frequencies
MD_LH_granger_all_frequencies
LH_mPFC_granger_all_frequencies
mPFC_LH_granger_all_frequencies
LH_vHPC_granger_all_frequencies
vHPC_LH_granger_all_frequencies
MD_mPFC_granger_all_frequencies
mPFC_MD_granger_all_frequencies
MD_vHPC_granger_all_frequencies
vHPC_MD_granger_all_frequencies
mPFC_vHPC_granger_all_frequencies
vHPC_mPFC_granger_all_frequencies
BLA-to-LH_granger_theta
BLA-to-LH_granger_gamma
LH-to-BLA_granger_theta
LH-to-BLA_granger_gamma
BLA-to-MD_granger_theta
BLA-to-MD_granger_gamma
MD-to-BLA_granger_theta
MD-to-BLA_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-mPFC_granger_gamma
mPFC-to-BLA_granger_theta
mPFC-to-BLA_granger_gamma
BLA-to-vHPC_granger_theta
BLA-to-vHPC_granger_gamma
vHPC-to-BLA_

In [72]:
for col in granger_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_granger_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["granger_timestamps"], items=x[col])[0], axis=1)

BLA_LH_granger_all_frequencies
LH_BLA_granger_all_frequencies
BLA_MD_granger_all_frequencies
MD_BLA_granger_all_frequencies
BLA_mPFC_granger_all_frequencies
mPFC_BLA_granger_all_frequencies
BLA_vHPC_granger_all_frequencies
vHPC_BLA_granger_all_frequencies
LH_MD_granger_all_frequencies
MD_LH_granger_all_frequencies
LH_mPFC_granger_all_frequencies
mPFC_LH_granger_all_frequencies
LH_vHPC_granger_all_frequencies
vHPC_LH_granger_all_frequencies
MD_mPFC_granger_all_frequencies
mPFC_MD_granger_all_frequencies
MD_vHPC_granger_all_frequencies
vHPC_MD_granger_all_frequencies
mPFC_vHPC_granger_all_frequencies
vHPC_mPFC_granger_all_frequencies
BLA-to-LH_granger_theta
BLA-to-LH_granger_gamma
LH-to-BLA_granger_theta
LH-to-BLA_granger_gamma
BLA-to-MD_granger_theta
BLA-to-MD_granger_gamma
MD-to-BLA_granger_theta
MD-to-BLA_granger_gamma
BLA-to-mPFC_granger_theta
BLA-to-mPFC_granger_gamma
mPFC-to-BLA_granger_theta
mPFC-to-BLA_granger_gamma
BLA-to-vHPC_granger_theta
BLA-to-vHPC_granger_gamma
vHPC-to-BLA_

In [73]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=granger_columns + ["granger_timestamps"], errors="ignore")

In [74]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

- Filtering power

In [75]:
TRIALS_AND_SPECTRAL_DF.head()

,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,competition_closeness,...,trial_mPFC-to-MD_granger_gamma,trial_MD-to-vHPC_granger_theta,trial_MD-to-vHPC_granger_gamma,trial_vHPC-to-MD_granger_theta,trial_vHPC-to-MD_granger_gamma,trial_mPFC-to-vHPC_granger_theta,trial_mPFC-to-vHPC_granger_gamma,trial_vHPC-to-mPFC_granger_theta,trial_vHPC-to-mPFC_granger_gamma,trial_granger_timestamps
0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,Subj 1 blocking Subj 2,...,"[0.2743068, 0.1816457, 0.20403126, 0.39949384,...","[0.4503957, 0.5514915, 0.20297746, 0.3768568, ...","[0.35033533, 0.25229087, 0.23366664, 0.3120289...","[0.3872143, 0.48870432, 0.45020568, 0.61337423...","[0.48877043, 0.22785409, 0.13890521, 0.265355,...","[0.48188347, 0.7108543, 0.18493713, 0.4764926,...","[0.34150958, 0.16291973, 0.20073977, 0.2023801...","[0.13133682, 0.23562104, 0.2440842, 0.15778197...","[0.35235053, 0.17450492, 0.37754112, 0.2303077...","[1130000, 1140000, 1150000, 1160000, 1170000, ..."
1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,Subj 2 Only,...,"[0.19837809, 0.5073394, 0.2601421, 0.17640117,...","[nan, 0.4103218, 0.1676341, nan, nan, 0.165973...","[nan, 0.41292667, 0.30698562, nan, nan, 0.3141...","[nan, 0.49083915, 0.09444503, nan, nan, 0.4753...","[nan, 0.24761999, 0.24060199, nan, nan, 0.3022...","[nan, 0.77709764, 0.46270016, nan, nan, 0.1674...","[nan, 0.4095989, 0.7093766, nan, nan, 0.181099...","[nan, 0.5647509, 0.45051277, nan, nan, 0.26605...","[nan, 0.18905884, 0.20349285, nan, nan, 0.2695...","[3530000, 3540000, 3550000, 3560000, 3570000, ..."
2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,Subj 2 blocking Subj 1,...,"[0.2451308, 0.22220398, 0.42297348, 0.21880822...","[0.3611453, 0.15535778, 0.10153123, 0.1033775,...","[0.28545213, 0.2011751, 0.30536118, 0.27875093...","[0.57998323, 0.70480734, 0.6927041, 0.79001415...","[0.26702482, 0.20771386, 0.6002271, 0.34293473...","[0.12613265, 0.34686637, 0.37530336, 0.0726977...","[0.2384008, 0.3289804, 0.38966492, 0.16582075,...","[0.33719257, 0.35507208, 0.390221, 0.41154325,...","[0.37617797, 0.284876, 0.4089172, 0.16548316, ...","[5830000, 5840000, 5850000, 5860000, 5870000, ..."
3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,Subj 2 Only,...,"[0.19989435, 0.3218986, 0.1846013, 0.44848886,...","[0.5495982, 0.15429461, 0.13154985, 0.21991509...","[0.43715674, 0.35011134, 0.31304544, 0.394448,...","[0.8319382, 0.8078395, 0.58001375, 0.2841426, ...","[0.35010985, 0.21684945, 0.14930977, 0.1355896...","[0.09518096, 0.20900358, 0.18624677, 0.2759102...","[0.1536734, 0.27427787, 0.33714557, 0.3097984,...","[0.49204752, 0.7473691, 0.5807837, 0.09881899,...","[0.3218193, 0.21725373, 0.33039397, 0.4509097,...","[7630000, 7640000, 7650000, 7660000, 7670000, ..."
4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,Subj 2 Only,...,"[0.25939098, 0.21315253, 0.46966016, 0.1842520...","[0.46183974, 0.16732047, 0.38939884, 0.481716,...","[0.3225777, 0.45540878, 0.3549677, 0.51858795,...","[0.14207345, 1.1679717, 1.0912386, 0.24064888,...","[0.24173824, 0.27463642, 0.2826689, 0.22011085...","[0.1101417, 0.14950544, 0.5411582, 0.23466948,...","[0.26624367, 0.34639746, 0.4196817, 0.29253903...","[0.21166928, 0.93035126, 0.45488098, 0.3350755...","[0.21460077, 0.40119368, 0.21195264, 0.1034163...","[8730000, 8740000, 8750000, 8760000, 8770000, ..."


In [76]:
power_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "power" in col and "timestamps" not in col and "calculation" not in col]

In [77]:
power_columns

['mPFC_RMS_filtered_power_all_frequencies_all_windows',
 'MD_RMS_filtered_power_all_frequencies_all_windows',
 'LH_RMS_filtered_power_all_frequencies_all_windows',
 'BLA_RMS_filtered_power_all_frequencies_all_windows',
 'vHPC_RMS_filtered_power_all_frequencies_all_windows',
 'mPFC_RMS_filtered_power_theta',
 'mPFC_RMS_filtered_power_gamma',
 'MD_RMS_filtered_power_theta',
 'MD_RMS_filtered_power_gamma',
 'LH_RMS_filtered_power_theta',
 'LH_RMS_filtered_power_gamma',
 'BLA_RMS_filtered_power_theta',
 'BLA_RMS_filtered_power_gamma',
 'vHPC_RMS_filtered_power_theta',
 'vHPC_RMS_filtered_power_gamma']

In [78]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

mPFC_RMS_filtered_power_all_frequencies
MD_RMS_filtered_power_all_frequencies
LH_RMS_filtered_power_all_frequencies
BLA_RMS_filtered_power_all_frequencies
vHPC_RMS_filtered_power_all_frequencies
mPFC_RMS_filtered_power_theta
mPFC_RMS_filtered_power_gamma
MD_RMS_filtered_power_theta
MD_RMS_filtered_power_gamma
LH_RMS_filtered_power_theta
LH_RMS_filtered_power_gamma
BLA_RMS_filtered_power_theta
BLA_RMS_filtered_power_gamma
vHPC_RMS_filtered_power_theta
vHPC_RMS_filtered_power_gamma


In [79]:
for col in power_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_power_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["power_timestamps"], items=x[col])[0], axis=1)

mPFC_RMS_filtered_power_all_frequencies
MD_RMS_filtered_power_all_frequencies
LH_RMS_filtered_power_all_frequencies
BLA_RMS_filtered_power_all_frequencies
vHPC_RMS_filtered_power_all_frequencies
mPFC_RMS_filtered_power_theta
mPFC_RMS_filtered_power_gamma
MD_RMS_filtered_power_theta
MD_RMS_filtered_power_gamma
LH_RMS_filtered_power_theta
LH_RMS_filtered_power_gamma
BLA_RMS_filtered_power_theta
BLA_RMS_filtered_power_gamma
vHPC_RMS_filtered_power_theta
vHPC_RMS_filtered_power_gamma


In [80]:
TRIALS_AND_SPECTRAL_DF["baseline_power_timestamps"]

0       [530000, 540000, 550000, 560000, 570000, 58000...
1       [2930000, 2940000, 2950000, 2960000, 2970000, ...
2       [5230000, 5240000, 5250000, 5260000, 5270000, ...
3       [7030000, 7040000, 7050000, 7060000, 7070000, ...
4       [8130000, 8140000, 8150000, 8160000, 8170000, ...
                              ...                        
1364    [60080000, 60090000, 60100000, 60110000, 60120...
1365    [61580000, 61590000, 61600000, 61610000, 61620...
1366    [63580000, 63590000, 63600000, 63610000, 63620...
1367    [65480000, 65490000, 65500000, 65510000, 65520...
1368    [66480000, 66490000, 66500000, 66510000, 66520...
Name: baseline_power_timestamps, Length: 1369, dtype: object

In [81]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=power_columns + ["power_timestamps"], errors="ignore")

In [82]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band
mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
start_frame
stop_frame
in_video_subjects
body_parts
box_top_left
box_top_right
reward_port
box_bottom_left
box_bottom_right
agent
base

# Filtering out phase

In [83]:
lfp_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "trace" in col and "timestamps" not in col and "calculation" not in col]

In [84]:
band_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "band" in col and "timestamps" not in col and "calculation" not in col]

In [85]:
phase_columns = [col for col in TRIALS_AND_SPECTRAL_DF.columns if "phase" in col and "timestamps" not in col and "calculation" not in col]

In [86]:
phase_columns = phase_columns + band_columns +lfp_columns

In [87]:
phase_columns

['mPFC_theta_phase',
 'MD_theta_phase',
 'LH_theta_phase',
 'BLA_theta_phase',
 'vHPC_theta_phase',
 'mPFC_gamma_phase',
 'MD_gamma_phase',
 'LH_gamma_phase',
 'BLA_gamma_phase',
 'vHPC_gamma_phase',
 'mPFC_theta_band',
 'MD_theta_band',
 'LH_theta_band',
 'BLA_theta_band',
 'vHPC_theta_band',
 'mPFC_gamma_band',
 'MD_gamma_band',
 'LH_gamma_band',
 'BLA_gamma_band',
 'vHPC_gamma_band']

In [88]:
for col in phase_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "baseline_{}".format(brain_region)
    updated_timestamp_col = "baseline_lfp_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)

mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band


In [89]:
for col in phase_columns:
    brain_region = col.replace("all_windows", "_").strip("_")
    print(brain_region)
    
    updated_item_col = "trial_{}".format(brain_region)
    updated_timestamp_col = "trial_lfp_timestamps".format(brain_region)
    TRIALS_AND_SPECTRAL_DF[updated_item_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[1], axis=1)
TRIALS_AND_SPECTRAL_DF[updated_timestamp_col] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["lfp_timestamps"], items=x[col])[0], axis=1)

mPFC_theta_phase
MD_theta_phase
LH_theta_phase
BLA_theta_phase
vHPC_theta_phase
mPFC_gamma_phase
MD_gamma_phase
LH_gamma_phase
BLA_gamma_phase
vHPC_gamma_phase
mPFC_theta_band
MD_theta_band
LH_theta_band
BLA_theta_band
vHPC_theta_band
mPFC_gamma_band
MD_gamma_band
LH_gamma_band
BLA_gamma_band
vHPC_gamma_band


In [90]:
TRIALS_AND_SPECTRAL_DF["baseline_lfp_timestamps"]

0       [526760, 526780, 526800, 526820, 526840, 52686...
1       [2926740, 2926760, 2926780, 2926800, 2926820, ...
2       [5226740, 5226760, 5226780, 5226800, 5226820, ...
3       [7026740, 7026760, 7026780, 7026800, 7026820, ...
4       [8126740, 8126760, 8126780, 8126800, 8126820, ...
                              ...                        
1364    [60073460, 60073480, 60073500, 60073520, 60073...
1365    [61573480, 61573500, 61573520, 61573540, 61573...
1366    [63573520, 63573540, 63573560, 63573580, 63573...
1367    [65473540, 65473560, 65473580, 65473600, 65473...
1368    [66473540, 66473560, 66473580, 66473600, 66473...
Name: baseline_lfp_timestamps, Length: 1369, dtype: object

In [91]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=phase_columns + ["lfp_timestamps"], errors="ignore")

In [92]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [93]:
for col in TRIALS_AND_SPECTRAL_DF:
    print(col)

tracked_subject
box_number
sleap_name
video_name
current_subject
tone_start_frame
reward_start_frame
tone_stop_frame
condition
competition_closeness
notes
experiment
session_dir
all_subjects
tone_start_timestamp
tone_stop_timestamp
trial_label
cohort
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
session_path
recording
subject
first_timestamp
last_timestamp
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
power_calculation_frequencies
coherence_calculation_frequencies
granger_calculation_frequencies
start_frame
stop_frame
in_video_subjects
body_parts
box_top_left
box_top_right
reward_port
box_bottom_left
box_bottom_right
agent
baseline_start_timestamp
baseline_stop_timestamp
baseline_agent_locations
baseline_agent_thorax_to_reward_port
baseline_agent_thorax_velocity
baseline_subject_locations
baseline_subject_thorax_to_reward_port
baseline_subject_thorax_velocity
baseline_video_frame
baseline_video_timestamps
trial_agent_locations
trial_agent_thorax_

# Filtering out spikes

In [95]:
raise ValueError()

ValueError: 

In [94]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"]).astype(int), axis=1)



KeyboardInterrupt



In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_spike_times"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: filter_spike_times(x["spike_times"], start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"]).astype(int), axis=1)


In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_start_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_stop_timestamp"].head()

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_spike_times"].iloc[0]

In [ ]:
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1], axis=1)
TRIALS_AND_SPECTRAL_DF["baseline_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["baseline_start_timestamp"], stop=x["baseline_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[1].T, axis=1)
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: utilities.helper.filter_by_timestamp_range(start=x["tone_start_timestamp"], stop=x["tone_stop_timestamp"], timestamps=x["neuron_average_timestamps"], items=x["neuron_average_fr"].T)[0], axis=1)

In [ ]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF.drop(columns=["spike_clusters", "spike_times", "neuron_average_fr", "neuron_average_timestamps",], errors="ignore")

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_fr"].iloc[0].shape

In [ ]:
TRIALS_AND_SPECTRAL_DF["trial_neuron_average_timestamp"].iloc[0].shape

# Filtering for rows that are in the video

In [96]:
TRIALS_AND_SPECTRAL_DF["in_video"] = TRIALS_AND_SPECTRAL_DF.apply(lambda x: x["start_frame"] <= x["trial_video_frame"][0] <= x["stop_frame"], axis=1)

In [97]:
TRIALS_AND_SPECTRAL_DF = TRIALS_AND_SPECTRAL_DF[TRIALS_AND_SPECTRAL_DF["in_video"]].reset_index()

In [98]:
TRIALS_AND_SPECTRAL_DF

,index,tracked_subject,box_number,sleap_name,video_name,current_subject,tone_start_frame,reward_start_frame,tone_stop_frame,condition,...,trial_LH_theta_band,trial_BLA_theta_band,trial_vHPC_theta_band,trial_mPFC_gamma_band,trial_MD_gamma_band,trial_LH_gamma_band,trial_BLA_gamma_band,trial_vHPC_gamma_band,trial_lfp_timestamps,in_video
0,0,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,1125,1225,1324,1.1,...,"[-0.1965594, -0.1906385, -0.1846614, -0.178647...","[-0.6278904, -0.6110051, -0.5924792, -0.572370...","[-0.86628234, -0.8033415, -0.73908955, -0.6737...","[-0.20077267, -0.27177534, -0.33255213, -0.378...","[0.042438794, -0.06269324, -0.16432026, -0.255...","[0.04900858, -0.041143205, -0.13162935, -0.216...","[-0.034259137, -0.07323087, -0.11172231, -0.14...","[0.091950886, 0.030192526, -0.039998747, -0.11...","[1126760, 1126780, 1126800, 1126820, 1126840, ...",True
1,1,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,3519,3619,3720,1.2,...,"[0.13085423, 0.18954182, 0.24687698, 0.3026803...","[0.23016661, 0.29850513, 0.36496305, 0.4293212...","[0.5561959, 0.5872191, 0.6153956, 0.6406603, 0...","[-0.20394863, -0.14480965, -0.0752808, -0.0018...","[-0.17063874, -0.1823823, -0.18196633, -0.1690...","[-0.13379243, -0.14699419, -0.15053916, -0.144...","[0.30445713, 0.2672235, 0.21840814, 0.16240521...","[0.41455534, 0.38904047, 0.34448272, 0.2843119...","[3526740, 3526760, 3526780, 3526800, 3526820, ...",True
2,2,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,5815,5915,6014,1.2,...,"[-0.117264144, -0.18494129, -0.25172737, -0.31...","[0.1510647, 0.102288105, 0.05333691, 0.0043895...","[0.15260983, 0.15515444, 0.15816545, 0.1616569...","[-0.2272427, -0.3109095, -0.375287, -0.4164722...","[-0.124851465, -0.1811587, -0.22630072, -0.257...","[-0.08437547, -0.14467272, -0.19493897, -0.231...","[-0.031209087, -0.07443029, -0.11561694, -0.15...","[-0.20006756, -0.29163465, -0.35794356, -0.393...","[5826740, 5826760, 5826780, 5826800, 5826820, ...",True
3,3,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,7612,7712,7811,1.2,...,"[0.571904, 0.57688946, 0.57978463, 0.5806063, ...","[1.093931, 1.0820043, 1.0669925, 1.0489453, 1....","[0.30607852, 0.26300424, 0.21877678, 0.1735299...","[-0.3201298, -0.3654427, -0.38641584, -0.38069...","[-0.22039057, -0.25678217, -0.2772813, -0.2792...","[-0.18128534, -0.2485002, -0.2972655, -0.32333...","[0.042083926, -0.04936665, -0.13645948, -0.211...","[0.16657004, 0.15574361, 0.14005733, 0.1210614...","[7626740, 7626760, 7626780, 7626800, 7626820, ...",True
4,4,"['1.1', '1.2']",1,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,1.1,8709,8809,8910,1.2,...,"[0.12631682, 0.12798116, 0.12979607, 0.1317504...","[-0.05114213, -0.075476125, -0.09903707, -0.12...","[-0.36225998, -0.3638156, -0.36478588, -0.3651...","[0.30167827, 0.2791766, 0.24319397, 0.19783372...","[0.16291839, 0.15559189, 0.14303625, 0.1267487...","[0.005076932, 0.04650006, 0.08811159, 0.127866...","[0.5704153, 0.55549455, 0.5117636, 0.44269294,...","[-0.14728308, -0.10256097, -0.05680496, -0.009...","[8726740, 8726760, 8726780, 8726800, 8726820, ...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
962,1364,"['1.2', '2.2']",2,20230630_115506_standard_comp_to_novel_agent_D...,20230630_115506_standard_comp_to_novel_agent_D...,1.2,60714,60814,60914,2.2,...,"[0.44909856, 0.4724813, 0.49443373, 0.51487184...","[0.3336614, 0.32942677, 0.32425243, 0.31815872...","[-0.36571392, -0.39123663, -0.41522902, -0.437...","[-0.019997114, -0.05471756, -0.08678063, -0.11...","[-0.15819319, -0.27901205, -0.3875353, -0.4765...","[0.016292952, -0.11031159, -0.23285471, -0

In [99]:
TRIALS_AND_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [ ]:
raise ValueError()